In [5]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [6]:
# 1) Load dataset ---
CSV_PATH = "spending_l9_dataset.csv"
df = pd.read_csv(CSV_PATH)
print("---- INITIAL HEAD OF DATA ----")
print(df.head())


---- INITIAL HEAD OF DATA ----
   CustomerID  Age  Income_$  SpendingScore  VisitsPerMonth  OnlinePurchases  \
0           1   28        33             78              14                9   
1           2   21        25             87               8               23   
2           3   23        24             88              13               10   
3           4   24        25             73              16               11   
4           5   20        23             88              17               16   

   Gender Region  
0  Female   East  
1    Male  North  
2    Male  South  
3  Female   West  
4    Male   West  


In [9]:
# 2) features OF clustering ----

Features = ['Age','Income_$', 'SpendingScore']
X = df[Features].copy()
print("\n=== FEATURES SNAPSHOT ===")
print(X.head())


=== FEATURES SNAPSHOT ===
   Age  Income_$  SpendingScore
0   28        33             78
1   21        25             87
2   23        24             88
3   24        25             73
4   20        23             88


In [10]:
 #Scaled features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("\nScaled shape:", X_scaled.shape)


Scaled shape: (200, 3)


In [11]:
for k in range(1, 11):
    km = KMeans(n_clusters=k, n_init="auto", random_state=42)
    km.fit(X_scaled)
    print(f"k={k} → SSE={km.inertia_:.2f}")

k=1 → SSE=600.00
k=2 → SSE=263.82
k=3 → SSE=112.13
k=4 → SSE=36.61
k=5 → SSE=34.30
k=6 → SSE=30.95
k=7 → SSE=27.52
k=8 → SSE=25.15
k=9 → SSE=23.59
k=10 → SSE=21.29


In [12]:
K = 4
kmeans = KMeans(n_clusters=K, n_init="auto", random_state=42)
labels = kmeans.fit_predict(X_scaled)

df["Cluster"] = labels.astype(int)
print(df['Cluster'])

0      2
1      2
2      2
3      2
4      2
      ..
195    3
196    3
197    3
198    3
199    3
Name: Cluster, Length: 200, dtype: int64


In [13]:
sil = silhouette_score(X_scaled, labels)
dbi = davies_bouldin_score(X_scaled, labels)
print(f"Silhouette Score : {sil:.3f} (closer to +1 is better)")
print(f"Davies–Bouldin   : {dbi:.3f} (lower is better)")

Silhouette Score : 0.697 (closer to +1 is better)
Davies–Bouldin   : 0.446 (lower is better)


In [21]:
centers_scaled = kmeans.cluster_centers_
centers_original = scaler.inverse_transform(centers_scaled)

centers_df = pd.DataFrame(centers_original, columns=Features)
centers_df.index.name = "Cluster"

print("CLUSTER CENTERS (Original Units)")
print(centers_df.round(2))

CLUSTER CENTERS (Original Units)
           Age  Income_$  SpendingScore
Cluster                                
0        33.66     56.32          53.58
1        56.58     28.92          19.60
2        22.38     24.14          83.10
3        37.40     99.16          79.24


In [20]:
#sanity check
sample_idx = [0, 1, 2]  # change to any row numbers
sanity = df.loc[sample_idx, Features + ["Cluster"]]
print("SANITY CHECK (3 Customers)")
print(sanity)

SANITY CHECK (3 Customers)
   Age  Income_$  SpendingScore  Cluster
0   28        33             78        2
1   21        25             87        2
2   23        24             88        2


In [23]:
OUT_PATH = "spending_labeled_clusters.csv"
df.to_csv(OUT_PATH, index=False)